In [ ]:

import kagglehub 

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)


In [ ]:
from pathlib import Path

path_to_dataset = Path("C:/Users/Lenovo/.cache/kagglehub/datasets/dylanjcastillo/7k-books-with-metadata/versions/3")


In [ ]:
import pandas as pd # type: ignore


In [ ]:
books=pd.read_csv(f"{path}/books.csv")

In [ ]:
books

In [ ]:
import seaborn as sns # type: ignore
import matplotlib.pyplot as plt # type: ignore

In [ ]:
ax=plt.axes()
sns.heatmap(books.isna().transpose(), cbar=False, cmap="YlGnBu")

plt.xlabel("columns")
plt.ylabel("missing values")

plt.show()

In [ ]:
import numpy as np

books["missing_desription"]=np.where(books["description"].isna(),1,0)
books["age_of_books"]=2024-books["published_year"]

In [ ]:
#create a list of subsets from colums we want from our book data set
columns_of_interest=["num_pages","age_of_books","missing_desription","average_rating"]
correlation_matrix=books[columns_of_interest].corr(method="spearman")
#spearman for non continous value
#convert into seabon heat map
sns.set_theme(style="white")
plt.figure(figsize=(8,6))
heatmap=sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm",cbar_kws={"label": "sperman correlation"})
heatmap.set_title("Correlation heatmap")
plt.show()


In [ ]:
#we are going to delete missing values so first thing first we gonna see how much /many of book will it cos
#conditional logic using pandas to find books that are going to have missing description,missing pages and missing average rating,missing publish year
missing_info_books = books[
    ~(books["description"].isna()) &
    ~(books["num_pages"].isna())  &
    ~(books["average_rating"].isna())  &
    ~(books["published_year"].isna())
]
missing_info_books

In [ ]:
missing_info_books["categories"].value_counts()
missing_info_books["categories"].value_counts().reset_index().sort_values(by="count", ascending=False)

In [ ]:
missing_info_books


In [ ]:
#This keeps the word count where there's text and leaves NaN where there's no description.
#applying a conditional transformation:
books["words_in_description"] = books["description"].apply(
    lambda x: len(x.split()) if isinstance(x, str) else np.nan
)

books


In [ ]:
#create an arbitrary cut point so a relatively straight forward way of working out where this cut off will be
#how many words a description needs to have in order to be meaningful is just come up with some cut_offs come up with some bins or how many words the description needs to have,inspect the descriptions that have those number of words
#create some pandas code to filter the pandas data frame so that we have all of those books where the words in description are between 1 and 4 to start ,so there's going to be our smallest group and we really just want to return the descriptions
# Step 1: Fill missing descriptions with empty string (so we can safely count words)
# Make a copy to safely work on it without warnings
missing_info_books = missing_info_books.loc[...]    #
missing_info_books["description"] = missing_info_books["description"].fillna("")
# Step 2: Create the 'words_in_description' column
missing_info_books["words_in_description"] = missing_info_books["description"].apply(lambda x: len(x.split()))
# Step 3: Filter books where description has between 1 and 4 words (inclusive)
short_descriptions = missing_info_books.loc[
    missing_info_books["words_in_description"].between(1, 4),
    "description"
]
short_descriptions
#bin1


In [ ]:
short_descriptions = missing_info_books.loc[
    missing_info_books["words_in_description"].between(5, 14),
    "description"
]
short_descriptions
#bin2

In [ ]:
short_descriptions = missing_info_books.loc[
    missing_info_books["words_in_description"].between(15, 24),
    "description"
]
short_descriptions
#bin3

In [ ]:
short_descriptions = missing_info_books.loc[
    missing_info_books["words_in_description"].between(25, 34),
    "description"
]
short_descriptions
#bin4

In [ ]:
#we are at a point where there is enough information,so it should be reasonable to use 25 words and up in the description as cut_off and now we see the impact of that is going to be
#create filtering code to clean up the data set again and remove all of those that have less than 25 in the description
#so we creaet a new data frame called book missing 25 words  and filter book missing and this will be all of those rows where word in description are more than equal to 25 and keeep those
# Ensure you're working on a clean copybut i am using loc to filter row and column
missing_info_books = missing_info_books.loc[...]

# Remove rows with less than 25 words in the description
books_missing_25_words = missing_info_books.loc[
    missing_info_books["words_in_description"] >= 25
]

In [ ]:
# Shows how many rows are left
books_missing_25_words

In [ ]:
"""
#different code works similarly just comment it to compare later on
# Make a copy to work safely
#books_missing_25_words = books_missing_25_words.copy()

# Fill NaNs in subtitle with empty string
#books_missing_25_words["subtitle"] = books_missing_25_words["subtitle"].fillna("")

# Create new column by combining title and subtitle (only include subtitle if it's non-empty)
books_missing_25_words["title_and_subtitle"] = books_missing_25_words.apply(
    lambda row: f"{row['title']}: {row['subtitle']}" if row["subtitle"].strip() else row["title"],
    axis=1
)

# Optional: check the result
books_missing_25_words
"""
#we want to create some final variable so cleaning and screening is completely done
#subtitle field:it's unusable on it's own because there is quite a lot of missing value but it's quite common to have a convention of books paired with title and subtitle
#so we create a new field called title and subtitle and pair the two if it existalso that where subtitle is missing we add the npw clause and we specify that when
# #also that where subtitle is missing we add the npw clause and we specify that when books missing and subtitle is missing use just the title but when it is present that is when there is a sub title what i want to do is to aggregate the title and the sub title
#and convert that explicitly to a string because there are lot of missing value and they are consider as a float in pandas we can join them using a colon
# Make a copy to work safely but i am using loc to filter row and column
missing_info_books = missing_info_books.loc[...]

# Create the 'title_and_subtitle' column using np.where
books_missing_25_words["title_and_subtitle"] = np.where(
    books_missing_25_words["subtitle"].notna() & books_missing_25_words["subtitle"].astype(str).str.strip().ne(""),
    books_missing_25_words["title"].astype(str) + ": " + books_missing_25_words["subtitle"].astype(str),
    books_missing_25_words["title"].astype(str)
)
# Optional: check the result
books_missing_25_words


In [ ]:
#create a new description column which tags each description with unique identifier i have 2 isbn13 and isbn 10 among which i will perform in isbn13,similar to title and subtitle ,create a new variable called tag description and take isbn 13 and description and do same as what i have done for title and subtitle
# Make a safe copy if needed but i am using loc to filter row and column
books_missing_25_words = books_missing_25_words.loc[...]

# Ensure isbn13 and description are strings, and handle missing description
books_missing_25_words["description"] = books_missing_25_words["description"].fillna("")

# Create the 'tagged_description' column
books_missing_25_words["tagged_description"] = books_missing_25_words["isbn13"].astype(str) + ": " + books_missing_25_words["description"]

# Optional: preview result
books_missing_25_words


In [ ]:
# Drop the specified columns
books_cleaned = books_missing_25_words.drop(columns=["missing_desription", "age_of_books", "subtitle"])

# Save the cleaned DataFrame to a CSV file
books_cleaned.to_csv("books_cleaned.csv", index=False)

# Optional: Check the result by loading and displaying the first few rows
books_cleaned
